In [1]:
!pip install transformers
from transformers import pipeline



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
generator = pipeline('text-generation', model = 'gpt2')
generator("I HATE MY JOB", max_length = 30, num_return_sequences=3)
## [{'generated_text': "Hello, I'm a language modeler. So while writing this, when I went out to meet my wife or come home she told me that my"},

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'I HATE MY JOB!!!!\n\nThe other day, I was doing my business at a mall in the North American city of Austin. I'},
 {'generated_text': "I HATE MY JOB BECAUSE IT SOE IS MUCKING ME BECAUSE I'm TOTALLY GARNY HILL"},
 {'generated_text': "I HATE MY JOB!!!\n\nCaveats\n\n-I don't know what to call this because I'm kind of into this"}]

## Idea 1
Train text generator on Amazon bad reviews


In [3]:
!pip install datasets
from datasets import load_dataset

dataset = load_dataset("amazon_us_reviews", "Digital_Video_Games_v1_00")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
import pandas as pd

data_df = pd.DataFrame.from_dict(dataset['train'])
data_df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,21269168,RSH1OZ87OYK92,B013PURRZW,603406193,Madden NFL 16 - Xbox One Digital Code,Digital_Video_Games,2,2,3,0,0,A slight improvement from last year.,I keep buying madden every year hoping they ge...,2015-08-31
1,US,133437,R1WFOQ3N9BO65I,B00F4CEHNK,341969535,Xbox Live Gift Card,Digital_Video_Games,5,0,0,0,1,Five Stars,Awesome,2015-08-31
2,US,45765011,R3YOOS71KM5M9,B00DNHLFQA,951665344,Command & Conquer The Ultimate Collection [Ins...,Digital_Video_Games,5,0,0,0,1,Hail to the great Yuri!,If you are prepping for the end of the world t...,2015-08-31
3,US,113118,R3R14UATT3OUFU,B004RMK5QG,395682204,Playstation Plus Subscription,Digital_Video_Games,5,0,0,0,1,Five Stars,Perfect,2015-08-31
4,US,22151364,RV2W9SGDNQA2C,B00G9BNLQE,640460561,Saints Row IV - Enter The Dominatrix [Online G...,Digital_Video_Games,5,0,0,0,1,Five Stars,Awesome!,2015-08-31


In [5]:
bad_reviews = data_df.loc[data_df["star_rating"] == 1]
len(bad_reviews)

24970

In [6]:
bad_reviews["review_body"].head()

10    As has been written by so many others, I quick...
14                Lames purchase I almost never made...
24                                               Gross!
26           Not the best graphics :(<br />Dissapointed
30                                               Stolen
Name: review_body, dtype: object

In [15]:
bad_reviews["message_len"]=[len(i) for i in bad_reviews["review_body"].values] # [0 for i in range(0,24970)]

<ipython-input-15-90594dcaec4e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bad_reviews["message_len"]=[len(i) for i in bad_reviews["review_body"].values] # [0 for i in range(0,24970)]


In [ ]:
# dall=dall.mul(row.loc[0],axis=1)

In [16]:
# [len(i) for i in bad_reviews["review_body"].values]

In [20]:
bad_reviews_small=bad_reviews.loc[bad_reviews["message_len"]<128]

In [21]:
bad_reviews_small.shape

(7671, 16)

In [22]:
bad_reviews.shape

(24970, 16)

In [19]:
# bad_reviews["message_len"]

10         540
14          37
24           6
26          42
30           6
          ... 
145415     108
145421     133
145425     240
145427     453
145429    2552
Name: message_len, Length: 24970, dtype: int64

In [23]:
# bad_reviews["message_len"]

14         37
24          6
26         42
30          6
35          5
         ... 
144841    100
144996     70
145177     77
145390     78
145415    108
Name: message_len, Length: 7671, dtype: int64

# Cleaning Data

In [7]:
# a little sentiment analysis
model_name = "finiteautomata/bertweet-base-sentiment-analysis"
sentiment_pipeline = pipeline("text-classification", model=model_name)

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


In [8]:
sentiment_pipeline.__dict__

{'task': 'text-classification',
 'model': RobertaForSequenceClassification(
   (roberta): RobertaModel(
     (embeddings): RobertaEmbeddings(
       (word_embeddings): Embedding(64001, 768, padding_idx=1)
       (position_embeddings): Embedding(130, 768, padding_idx=1)
       (token_type_embeddings): Embedding(1, 768)
       (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
       (dropout): Dropout(p=0.1, inplace=False)
     )
     (encoder): RobertaEncoder(
       (layer): ModuleList(
         (0-11): 12 x RobertaLayer(
           (attention): RobertaAttention(
             (self): RobertaSelfAttention(
               (query): Linear(in_features=768, out_features=768, bias=True)
               (key): Linear(in_features=768, out_features=768, bias=True)
               (value): Linear(in_features=768, out_features=768, bias=True)
               (dropout): Dropout(p=0.1, inplace=False)
             )
             (output): RobertaSelfOutput(
               (dense): Line

In [25]:
review = bad_reviews_small["review_body"].values[0]
sentiment_pipeline(review)

[{'label': 'NEG', 'score': 0.9216219782829285}]

In [ ]:
cleaner_data = None

for index, bad_review in bad_reviews_small.iterrows():
    sentiment = sentiment_pipeline(bad_review["review_body"])[0]
    if (sentiment["label"] == "POSITIVE"):
        cleaner_data = bad_reviews.drop(index=index, axis = 0)
    elif sentiment["score"] < 0.8:
        cleaner_data = bad_reviews.drop(index=index, axis = 0)


In [ ]:
len(cleaner_data )

# Retraining on top of transformers

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model = AutoModelForCausalLM.from_pretrained("gpt2")

In [ ]:
bad_reviews_array = bad_reviews["review_body"].values.tolist()
bad_reviews_array[0:2]

In [ ]:
tokenized_bad_reviews = tokenizer(bad_reviews_array, padding=True, truncation=True, max_length=512, return_tensors="pt")

In [ ]:
type(tokenized_bad_reviews)

In [ ]:
# 
from transformers import Trainer, TrainingArguments
import torch

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_bad_reviews,
    data_collator=lambda data: {"input_ids": torch.stack([f[0] for f in data]), "attention_mask": torch.stack([f[1] for f in data]), "labels": torch.stack([f[0] for f in data])},
)

trainer.train()